# Sensor Override Attack Example

In [ ]:
from IPython.display import display, HTML
display(HTML('<a target="_blank" href="https://colab.research.google.com/github/WaterFutures/EPyT-Flow/blob/main/docs/examples/sensor_override_attack.ipynb"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>'))

This example demonstrastes how to add a sensor overriding attack to a scenario -- more information can be found in the [documentation](https://epyt-flow.readthedocs.io/en/stable/tut.events.html#sensor-reading-attacks).

In [ ]:
%pip install epyt-flow --quiet

In [ ]:
import numpy as np
from epyt_flow.data.benchmarks import load_leakdb_scenarios
from epyt_flow.simulation import ScenarioSimulator, SENSOR_TYPE_LINK_FLOW
from epyt_flow.simulation.events import SensorOverrideAttack
from epyt_flow.utils import to_seconds, plot_timeseries_data

Load the first Hanoi scenario in [LeakDB](https://epyt-flow.readthedocs.io/en/stable/epyt_flow.data.benchmarks.html#epyt_flow.data.benchmarks.leakdb.load_scenarios):

In [ ]:
config = load_leakdb_scenarios(scenarios_id=["1"], use_net1=False, verbose=False)[0]

In [ ]:
sim = ScenarioSimulator(scenario_config=config)

Set simulation duration to two days:

In [ ]:
sim.set_general_parameters(simulation_duration=to_seconds(days=2))

Override the sensor readings of the flow sensor at link "1" with the value "42" for 2hrs -- i.e. time steps 10 - 15.

This is done by creating an instance of the [SensorOverrideAttack](https://epyt-flow.readthedocs.io/en/stable/epyt_flow.simulation.events.html#epyt_flow.simulation.events.sensor_reading_attack.SensorOverrideAttack) class and adding this sensor reading event to the scenario by calling [add_sensor_reading_event()](https://epyt-flow.readthedocs.io/en/stable/epyt_flow.simulation.html#epyt_flow.simulation.scenario_simulator.ScenarioSimulator.add_sensor_reading_event):

In [ ]:
new_sensor_values = np.array([42]*5)
sensor_override_attack = SensorOverrideAttack(new_sensor_values,
                                              start_time=to_seconds(hours=5),
                                              end_time=to_seconds(hours=7),
                                              sensor_id="1",
                                              sensor_type=SENSOR_TYPE_LINK_FLOW)

sim.add_sensor_reading_event(sensor_override_attack)

Run simulation and and plot flow readings:

In [ ]:
scada_data = sim.run_simulation()

In [ ]:
scada_data.plot_flows(sensor_locations=["1"])

Sensor readings WITHOUT the sensor overriding attack:

In [ ]:
# Remove attack
scada_data.sensor_reading_events = []

# Recompute and show final sensor readings
scada_data.plot_flows(sensor_locations=["1"])

Do not forget to close the simulation!

In [ ]:
sim.close()